# Instalar biblioteca Biopython

In [1]:
pip install biopython

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 16.2 MB/s eta 0:00:00


# Importar bibliotecas para o desenvolvimento do projeto

In [2]:
import Bio
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord

# Fazer upload do arquivo fasta reads4

In [3]:
from google.colab import files
uploaded = files.upload()

Saving reads4.fasta to reads4.fasta


# A partir de 4 sequências curtas de DNA sequenciado, arquivo "reads.fasta" , fazer um código que monte o Contig correspondente.

## Armazenar as 4 sequências de DNA do arquivo fasta

In [4]:
DNA_list = []
for record in SeqIO.parse("reads4.fasta", "fasta"):
  Seq = record.seq
  DNA_list.append(str(Seq))

for DNA in DNA_list:
  print(f'DNA: {DNA}')

DNA: ATTAGACCTG
DNA: CCTGCCGGAA
DNA: AGACCTGCCG
DNA: GCCGGAATAC


## Criação de funções auxiliares para o desenvolvimento do código

In [5]:
def get_chunk_DNA(DNA, min, max):
  return DNA[min:max]

def get_index_DNA(DNA, seq):

  index = 0
  while (len(seq) + index < len(DNA)):
    
    search_seq = DNA[index:len(seq) + index]
    if seq in search_seq:
      return (index)

    index += 1

  return -1

## Aplicar algoritmo guloso para montagem de Genoma

In [6]:
indexes_list = [0]
common_DNA_list = []
genome = ""

min = 0
max = 3
start = min

while True:
  
  ## Verifica se existem nucleotideos a serem lidos na sequencia atual
  if (max > len(DNA_list[indexes_list[0]])):

    ## Se nao existir DNA em comum, finaliza o algoritmo e 
    ## insere o restante da sequencia no genoma montado
    if (len(common_DNA_list) == 0):
      non_common_seq = DNA_list[indexes_list[0]][start:max]
      genome += (non_common_seq)
      break

    common_seq = common_DNA_list[-1]
    non_common_seq = DNA_list[indexes_list[0]][start:min]
    genome += (non_common_seq + common_seq)

    indexes_list.pop(0)
    index = get_index_DNA(DNA_list[indexes_list[0]], common_seq)
    min = index + len(common_seq)
    max = min + 3

    start = min
    common_DNA_list = []

  ## Ainda existem nucleotideos que nao foram lidos na sequencia atual
  else:

    current_DNA = get_chunk_DNA(DNA_list[indexes_list[0]], min, max)
    findCommonSeq = False
    DNA_index = 0
    
    for DNA in DNA_list:
      
      ## Verifica semelhanca do pedaco de DNA (current_DNA) 
      ## entre as outras demais sequencias de DNA da lista
      if (DNA_index != indexes_list[0] and current_DNA in DNA):

          common_DNA_list.append(current_DNA)
          findCommonSeq = True

          if (DNA_index not in indexes_list):
            indexes_list.append(DNA_index)

          break

      DNA_index += 1

    max += 1
    if (not findCommonSeq):
      min += 1

print(f'Genoma Montado: {genome}')

Genoma Montado: ATTAGACCTGCCGGAATAC


## Salvar o contig em arquivo no formato FASTA.

In [7]:
id_content = "contig"
description_content = ""

content = SeqRecord(
    Bio.Seq.Seq(genome),
    id = id_content,
    description = description_content
)

SeqIO.write(
    content,
    "contig.fasta",
    "fasta"
)

1